<div style="text-align: center; margin: 50px">

<h1 style="color: darkblue; background-color: white; text-align: center;">Qubit by Qubit - Semester 2</h1>
<h3>Quantum Teleporation HW 18</h3>
</div>

#### Quantum Mechanics allows to do some super cool things, one of those is quantum teleportation.!

First let's clarify what is being teleported - we are transfering a quantum state rather than an object of some sort. We can transfer $|\Psi \rangle$ from Alice to Bob without copying (cloning) the state $|\Psi \rangle$ and only sending two classical bits of information.

In this homework we are going to work through the steps of creating a teleportation circuit that will send a state $|\Psi\rangle$ from Alice to Bob but also allow us to see the states of each of the qubits thanks to the wonders of simulation.

In order to do create the circuit in a way that allows us to visualise teleportation across the qubits we need to use some fairly niche parts of qiskit. To save you from going through the depths of source code and/or documentation we have **largely completed this notebook for you**! It's your job to complete only a few functions and run everything, don't worry too much about any unusual syntax!

Try instead to really understand what is going on at each point, feel free to add you own cells and test things out.

In [1]:
# As usual import qiskit as q
import qiskit as q
import numpy as np
import matplotlib.pyplot as plt

## Activity 1: Quantum teleportation pre-requisites

A register is just a collection of bits, they can either be quantum or classical as shown below. They make the creation of quantum circuits easier. Let's learn how to make them! For this excercise we need to create two seperate single bit classical registers (this is non-standard qiskit but makes for a great demonstration)

1. Let's create a quantum register named qr with 3 qubits and give it a label 'q'

`qr = q.QuantumRegister(3,'q')`

2. Let's create two classical registers, one named "c_0" and another named "c_1" each with 1 classical bit

`c_0 = q.ClassicalRegister(1, 'c0')`
`c_1 = q.ClassicalRegister(1, 'c1')`

3. We can create a quantum circuit named qc, with a quantum register named qr with 3 qubits and a classical register named cr with 2 classical bits.

`qc = q.QuantumCircuit(qr, cr_0, cr_1)`

### 1a) Create a quantum circuit using the method described as above:
Apply measurement gates on the $q_0$ to $c_0$ and the $q_1$ to $c_1$
N.B. This should look like an empty circuit with only measurement gates

In [ ]:
# TODO

## Conditional gates
For us to correctly create the circuit we need to use a qiskit feature where we apply a controlled gates that depends on the state of a classical bit for the control:
```
QuantumCircuit.z(2).c_if(cr_0, 1)
```
This means we apply a Z gate to qubit 2 only if classical bit 0 is equal to 1
### 1b) Place a conditional z gate on the $Q_2$ qubit condtioned on classical $C_1$ on the circuit above that applies only if the $C_1 = 1$.

In [ ]:
# TODO

# Activity 2: State teleportation!

In this experiment we are going to imagine we have two people, Alice and Bob. Alice has a state $|\Psi \rangle$ that they want to send to Bob. We are going to imagine that Alice has qubit $0$ and Bob has qubit $2$ show a state $|\Psi \rangle$ teleporting from qubit $0$ to qubit $2$ on our circuit.

In order to teleport the state $|\Psi \rangle$, we must create an entagled pair of qubits and give one of each of the qubits to Alice and Bob. We can imagine a third person Charlie giving a pair of entangled qubits to each of Alice and Bob , this is going to be qubit $1$ and $2$.

Confused? Let me clarify:

Alice starts with $Q_0$,
Charlie creates and entagled pair: $Q_1$ and $Q_2$
Charlie then distritbutes $Q_1$ to Alice and $Q_2$ to Bob

At this point our teleportation circuit if prepared!

In order for Alice to send the state $|\Psi \rangle$ of $Q_0$, they only need to send two classical bits of data to Bob and bob will be able to completely recreate the state $|\Psi \rangle$ on his $Q_2$!

Amazing! But there's two more key parts we need to figure out: Alice needs to know what two classical bits to send to Bob, and Bob needs to know how to interpret those classical bit's as intructions to recreate $|\Psi \rangle$.

### Let's first define a function that creates our abstracted teleportation protocol, we've done this for you:

In [4]:
# Teleportation Protocol:

def teleportation_circuit(psi):

    # First we make our state Psi into a initialisation gate
    init_gate = q.extensions.Initialize(psi)
    init_gate.label = "Secret Message!"
    
    # Create our circuit!
    qr = q.QuantumRegister(3, name="q")   # Create a 3 qubit quantum register
    c_0 = q.ClassicalRegister(1, name="c_0") # Create 2 single bit classical registers
    c_1 = q.ClassicalRegister(1, name="c_1")
    qc = q.QuantumCircuit(qr, c_0, c_1)

    # Initialise Alice's Qubit to have state Psi
    qc.append(init_gate, [0])
    # We use barrier to split up each segment
    qc.barrier()
    
    # Use snapshot magic to record the inial statevector
    qc.snapshot('1')
    # We use barrier to split up each segment
    qc.barrier()

    # Charlie creates the Bell pair
    create_bell_pair(qc, 1, 2)
    # We use barrier to split up each segment
    qc.barrier()

    # Alice does gate prep
    alice_gates(qc, 0, 1)
    # We use barrier to split up each segment
    qc.barrier()

    # Alice measures her state and sends to Bob
    measure_and_send(qc, 0, 1)
    # We use barrier to split up each segment
    qc.barrier()

    # Bob decodes the classical bits and re-creates Psi
    bob_gates(qc, 2, c_0, c_1)
    # We use barrier to split up each segment
    qc.barrier()
    
    # Use snapshot magic to record the final statevector
    qc.snapshot('2')  

    # We want our function to return a fully prepared circuit
    return qc

# Alice, Bob, and Charlie steps:


Charlie needs to make an entangled pair - we are going to create the $|\Phi^+\rangle$ state between the $q_1$ and $q_2$ qubits, where:

$$
| \Phi^+ \rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle
$$

Alice is now going to apply a CNOT gate with $Q_0$ as control and $Q_1$ as target as well as a hadamard to $Q_0$.

Then Alice will measure the $Q_0$ and $Q_1$ qubits and send the classical values to Bob via the classical register.

Now with only two bits of classical information, Bob can completely recreate the state of $| \Psi \rangle$ on his end!

To do this he can follow the following format:

**If the first bit is 1, he applies a Z gate, if the second bit is 1 he applies an X gate.**

00 = Nothing

01 = $X$ gate

10 = $Z$ gate

11 = $ZX$ gate

## 2a) Complete the `create_bell_pair()` and `alice_gate()` functions:
Use the `measure_and_send()` and `bob_gates()` as templates!

In [5]:
def create_bell_pair(qc, a, b):
    # Here we want to create the Phi+ bell state
    # 1. Place a hadamard gate on 'a'
    # 2. Place a CNOT gate with 'a' as control and 'b' as target
    
    # TODO

In [6]:
def alice_gates(qc, psi, a):
    # 1. Place a CNOT gate with Psi as control and 'a' as target
    # 2. Place a hadamard gate on Psi
    
    # TODO

In [7]:
def measure_and_send(qc, a, b):
    # Measure a to 0 and b to 1
    qc.measure(a,0)
    qc.measure(b,1)

In [8]:
def bob_gates(qc, qubit, crz, crx):
    # Here we use c_if to control our gates with a classical
    # bit instead of a qubit
    qc.x(qubit).c_if(crx, 1) # Apply gates if the registers 
    qc.z(qubit).c_if(crz, 1) # are in the state '1

# Activity 3: Simulating Quantum Teleportation!

Let's run a simulation on the `statevector_simulator`. We've created Alice's $| \Psi \rangle$ state for you:

In [9]:
psi = np.array([0.5533920757991503+0.3043529040180291j, 0.6147796854942953+0.4724113234904887j])

Now we can create a teleporter circuit using the teleportation_circuit function we defined above:

`circ = teleportation_circuit(psi)`

### 3a) Create a `teleportation_circuit()` object called `circ` and draw it:

In [13]:
# Make a teleportation circuit called circ here: 

# TODO

Now we can simulate it using the statvector_simulator:

In [11]:
# Execute the circuit on the simulated backend
backend = q.Aer.get_backend('statevector_simulator')
result = q.execute(circ, backend).result()

# Put our snapshots into a list
snapshots = result.data()['snapshots']['statevector']

# Get Alice's state:
alice_state = snapshots['1']

# Get Bob's final state:
bob_state = snapshots['2']

### Use this cell to check if you've implemented you circuit correctly!
Running this cell will print out Alice and Bob's states from the start and finish of the experiment!

In [ ]:
def check_same_state(alice, bob, psi):
    a = alice[0]
    b = [c for c in bob[0] if c != 0]
    print("Psi state: ", psi[0], psi[1])
    print("Alice's state: ", a[0], a[1])
    print("Bob's state: ", b[0], b[1])
    # For simplicity I am rounding and summing to check for equality
    if np.round(a[0]+a[1], 5) == np.round(b[0]+ b[1], 5):
        print("State Successfully Teleported!")
    else:
        print("Error: Bob did not get the right state!")

# Check if it's the same state
check_same_state(alice_state, bob_state, psi)

### © 2020 The Coding School

**All rights reserved**

*Use of this activity is for personal use only. Copying, reproducing, distributing, posting or sharing this activity in any manner with any third party are prohibited under the terms of this registration. All rights not specifically licensed under the registration are reserved.*